<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MCP%26A2A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime

# --- 1. Model Context Protocol Representation ---
# This class simulates how an agent maintains its internal state/context.
class AgentContext:
    def __init__(self, agent_name):
        self.agent_name = agent_name
        self.history = []
        self.current_state = {}

    def update_context(self, new_info):
        """Updates the current state and adds to history."""
        self.history.append((datetime.datetime.now(), new_info))
        self.current_state.update(new_info)
        print(f"[{self.agent_name} Context]: Updated with: {new_info}")

    def get_current_context(self):
        """Returns the current relevant context."""
        return self.current_state.copy()

    def clear_context(self):
        """Resets the context for a new interaction."""
        self.history = []
        self.current_state = {}
        print(f"[{self.agent_name} Context]: Cleared.")

# --- 2. Agent-to-Agent Interaction Simulation ---

class BaseAgent:
    def __init__(self, name):
        self.name = name
        self.context = AgentContext(name)
        self.connections = {} # To simulate knowing other agents

    def connect_to(self, agent):
        """Simulates establishing a connection to another agent."""
        self.connections[agent.name] = agent
        print(f"[{self.name}]: Connected to {agent.name}")

    def send_message(self, recipient_name, message):
        """Simulates sending a message to another agent."""
        if recipient_name in self.connections:
            print(f"\n[{self.name} -> {recipient_name}]: Sending message: {message}")
            return self.connections[recipient_name].receive_message(self.name, message)
        else:
            print(f"[{self.name}]: Error: No connection to {recipient_name}")
            return {"status": "error", "message": f"Agent {recipient_name} not found."}

    def receive_message(self, sender_name, message):
        """Placeholder for message reception and processing."""
        print(f"[{self.name}]: Received message from {sender_name}: {message}")
        # Agents would typically update their context here based on the message
        self.context.update_context({"last_received_message": message, "from_agent": sender_name})
        return {"status": "acknowledged", "agent": self.name}

# --- Specific Agents for Flight Planning Scenario ---

class UserInterfaceAgent(BaseAgent):
    def __init__(self):
        super().__init__("UserInterfaceAgent")

    def initiate_flight_search(self, origin, destination, date):
        user_request = {"origin": origin, "destination": destination, "date": date}
        self.context.update_context({"user_flight_request": user_request})
        print(f"[{self.name}]: User request received and stored in context.")

        # Agent-to-Agent Interaction: Send request to FlightSearchAgent
        response = self.send_message(
            "FlightSearchAgent",
            {"type": "search_flight", "details": user_request}
        )
        if response and response.get("status") == "success":
            self.context.update_context({"flight_search_results": response.get("data")})
            print(f"[{self.name}]: Flight search results received and stored in context.")
            print("\n--- Final Information for User ---")
            print(f"Flight from {origin} to {destination} on {date}:")
            print(f"  Options: {response['data'].get('flights', 'No flights found.')}")
            print(f"  Weather at {destination}: {response['data'].get('weather', 'N/A')}")
        else:
            print(f"[{self.name}]: Failed to get flight search results: {response}")

class FlightSearchAgent(BaseAgent):
    def __init__(self):
        super().__init__("FlightSearchAgent")

    def receive_message(self, sender_name, message):
        super().receive_message(sender_name, message) # Update internal context with message
        if message.get("type") == "search_flight":
            details = message.get("details")
            self.context.update_context({"current_flight_search_params": details})
            print(f"[{self.name}]: Processing flight search for {details['origin']} to {details['destination']}.")

            # Simulate searching for flights
            simulated_flights = [
                {"flight_number": "FL101", "airline": "AirAI", "price": "$250"},
                {"flight_number": "BB202", "airline": "BotAirlines", "price": "$280"}
            ]

            # Agent-to-Agent Interaction: Get weather from WeatherAgent
            weather_response = self.send_message(
                "WeatherAgent",
                {"type": "get_weather", "location": details["destination"], "date": details["date"]}
            )
            weather_info = "Weather unavailable."
            if weather_response and weather_response.get("status") == "success":
                weather_info = weather_response.get("data", {}).get("forecast", "Clear")
                self.context.update_context({"destination_weather": weather_info})
                print(f"[{self.name}]: Received weather information for {details['destination']}.")

            return {"status": "success", "data": {"flights": simulated_flights, "weather": weather_info}}
        return {"status": "error", "message": "Unknown message type."}

class WeatherAgent(BaseAgent):
    def __init__(self):
        super().__init__("WeatherAgent")

    def receive_message(self, sender_name, message):
        super().receive_message(sender_name, message) # Update internal context with message
        if message.get("type") == "get_weather":
            location = message.get("location")
            date = message.get("date")
            print(f"[{self.name}]: Looking up weather for {location} on {date}.")

            # Simulate weather lookup
            if "Paris" in location:
                forecast = "Rainy with a chance of sunshine (simulated)."
            elif "New York" in location:
                forecast = "Cloudy (simulated)."
            else:
                forecast = "Sunny (simulated)."

            self.context.update_context({"last_weather_query": {"location": location, "forecast": forecast}})
            return {"status": "success", "data": {"location": location, "date": date, "forecast": forecast}}
        return {"status": "error", "message": "Unknown message type."}

# --- Simulation Execution ---

if __name__ == "__main__":
    print("--- Setting up AI Agents for Flight Planning ---")
    user_agent = UserInterfaceAgent()
    flight_agent = FlightSearchAgent()
    weather_agent = WeatherAgent()

    # Establish Agent-to-Agent connections
    user_agent.connect_to(flight_agent)
    flight_agent.connect_to(weather_agent)

    print("\n--- Initiating a Flight Search Scenario ---")
    user_agent.initiate_flight_search("Montreal", "Paris", "2025-07-15")

    print("\n--- Reviewing Agent Contexts After Interaction ---")
    print(f"\n{user_agent.name} Current Context: {user_agent.context.get_current_context()}")
    print(f"\n{flight_agent.name} Current Context: {flight_agent.context.get_current_context()}")
    print(f"\n{weather_agent.name} Current Context: {weather_agent.context.get_current_context()}")

    print("\n--- Initiating another Flight Search to observe context change ---")
    user_agent.initiate_flight_search("New York", "London", "2025-08-20")

    print("\n--- Reviewing Agent Contexts After Second Interaction ---")
    print(f"\n{user_agent.name} Current Context: {user_agent.context.get_current_context()}")
    print(f"\n{flight_agent.name} Current Context: {flight_agent.context.get_current_context()}")
    print(f"\n{weather_agent.name} Current Context: {weather_agent.context.get_current_context()}")

    # Demonstrate clearing context
    print("\n--- Demonstrating Context Clearing ---")
    user_agent.context.clear_context()
    print(f"\n{user_agent.name} Current Context After Clear: {user_agent.context.get_current_context()}")

--- Setting up AI Agents for Flight Planning ---
[UserInterfaceAgent]: Connected to FlightSearchAgent
[FlightSearchAgent]: Connected to WeatherAgent

--- Initiating a Flight Search Scenario ---
[UserInterfaceAgent Context]: Updated with: {'user_flight_request': {'origin': 'Montreal', 'destination': 'Paris', 'date': '2025-07-15'}}
[UserInterfaceAgent]: User request received and stored in context.

[UserInterfaceAgent -> FlightSearchAgent]: Sending message: {'type': 'search_flight', 'details': {'origin': 'Montreal', 'destination': 'Paris', 'date': '2025-07-15'}}
[FlightSearchAgent]: Received message from UserInterfaceAgent: {'type': 'search_flight', 'details': {'origin': 'Montreal', 'destination': 'Paris', 'date': '2025-07-15'}}
[FlightSearchAgent Context]: Updated with: {'last_received_message': {'type': 'search_flight', 'details': {'origin': 'Montreal', 'destination': 'Paris', 'date': '2025-07-15'}}, 'from_agent': 'UserInterfaceAgent'}
[FlightSearchAgent Context]: Updated with: {'curre